<a href="https://colab.research.google.com/github/Zak1105/Trimble-test/blob/main/Trimble_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip dataset.zip


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ExponentialLR
from tqdm import tqdm
import random
# Define a transformation to resize images to 224x224 and perform data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the data from the dataset folder
dataset = ImageFolder("dataset", transform=transform)

# Split the data into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
"""
# Count the number of samples in each class

count_class_0 = sum(labels == 0 for _, labels in train_dataset)
count_class_1 = len(train_dataset) - count_class_0
print(count_class_0)
print(count_class_1)
# Calculate the upsampling factor

undersampled_train_dataset= []
# Échantillons aléatoires de la classe majoritaire (classe 1)
for inputs, labels in train_dataset:
    if labels == 1 and random.random() < (count_class_0 / count_class_1):
        undersampled_train_dataset.append((inputs, labels))"""
batch_size=16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=4)

# Define the model ResNet101 with an output for classification
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = torch.hub.load('pytorch/vision', 'resnet101', pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 1)  # Output layer with one outputs
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        return self.sigmoid(self.resnet(x))

# Instantiate the model
model = ResNetModel()

device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion =nn.L1Loss().to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.002)
# Define the scheduler for optimisate the learning rate
scheduler = ExponentialLR(optimizer, gamma=0.9,verbose=True)
# Training the model

model.to(device)

epochs = 10
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
print ("---------------------------------- Start of training ----------------------------------")
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train_samples = 0

    for inputs, labels in tqdm(train_loader):
        labels=labels.to(device)

        inputs, labels = inputs.to(device), labels.unsqueeze(1).float() #
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        outputs[outputs>0.5]=1
        outputs[outputs<0.5]=0

        running_loss += loss.item()

        correct_train += (outputs == labels).sum().item()
        total_train_samples += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train /total_train_samples
    scheduler.step()

    # Validation
    model.eval()
    running_loss = 0.0
    correct_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            labels=labels.to(device)
            inputs, labels = inputs.to(device), labels.unsqueeze(1).float()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            running_loss += loss.item()
            outputs[outputs>0.5]=1
            outputs[outputs<0.5]=0

            correct_val += (outputs == labels).sum().item()

    val_loss = running_loss / len(val_loader)
    val_accuracy = 100 * correct_val / len(val_dataset)

    print(f"Epoch {epoch + 1}/{epochs}, "
          f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%")

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
print("---------------------------------- End of trainning ----------------------------------")

plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs + 1), train_losses, label='Train')
plt.plot(range(1, epochs + 1), val_losses, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Train and Validation Loss')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs + 1), train_accuracies, label='Train')
plt.plot(range(1, epochs + 1), val_accuracies, label='Validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Train and Validation Accuracy')
plt.show()


In [ ]:
!unzip /content/test.zip

In [ ]:
torch.save(model,'model_final.pth')


In [ ]:

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.eval()
test_dataset = ImageFolder("test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

# Create prediction folders
if not os.path.exists("predicted_0"):
    os.makedirs("predicted_0")
if not os.path.exists("predicted_1"):
    os.makedirs("predicted_1")

with torch.no_grad():
    for idx, (inputs, _) in enumerate(test_loader):
        inputs =  inputs.to(device)
        outputs = model(inputs)
        predicted = 0 if outputs.item() < 0.5 else 1
        print(f"Image {idx}: Predicted {'road' if predicted == 1 else 'field'}")

        original_image = inputs.permute(0, 2, 3, 1).cpu().numpy().squeeze()

        # Save the image with appropriate naming
        prediction_folder = "predicted_1" if predicted == 1 else "predicted_0"
        image_name = f"image_{idx}_predicted_{predicted}.png"
        image_path = os.path.join(prediction_folder, image_name)

        plt.imshow(original_image, cmap='gray')
        plt.title(f"Predicted: {'road' if predicted == 1 else 'field'}")
        plt.savefig(image_path)
        plt.clf()  # Clear the plot for the next iteration




